In [1]:
import pandas as pd

df = pd.read_csv('cleaned_airbnb_scalleddown.csv')


In [2]:
import json
import pickle

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Print the shape and number of missing data per column.
print(f'This dataframe has {df.shape[0]} entries and {df.shape[1]} columns.')
print('----------')
print('Total missing entries per column:\n')
print(df.isna().sum())

# Print the head of the dataframe.
print('----------')
df.head()

This dataframe has 31999 entries and 12 columns.
----------
Total missing entries per column:

space                   0
description             0
city                    0
state                   0
room_type               0
price                   0
security_deposit        0
guests_included         0
extra_people            0
minimum_nights          0
maximum_nights          0
review_scores_rating    0
dtype: int64
----------


,space,description,city,state,room_type,price,security_deposit,guests_included,extra_people,minimum_nights,maximum_nights,review_scores_rating
0,"I like to call my place ""the penthouse"". It's ...",I like to think of my place as a cozy escape. ...,BROOKLYN,NY,Entire home/apt,87,0,1,0,4,1125,100
1,This room is one of two bedrooms open to book...,"Gorgeous, spacious, clean & cozy bedroom in th...",NEW YORK,NY,Private room,100,0,1,25,2,25,96
2,Our craftsman style home was aesthetically rem...,This cozy two bedroom apt is well lit. Then ap...,SEATTLE,WA,Entire home/apt,175,250,2,10,3,1125,100
3,There are two bathrooms on this floor where th...,The subways that is near with the house is the...,BROOKLYN,NY,Private room,80,0,2,0,30,1125,100
4,- Just renovated 3.5 bedroom / 2 bathroom pent...,Just updated 3.5 BR | 2 BA condo that features...,BOSTON,MA,Entire home/apt,600,650,5,40,2,270,99


In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, random_state=42)

train.shape, test.shape

((22399, 12), (9600, 12))

In [10]:
train, val = train_test_split(train, train_size=0.80, test_size=0.20,
                               random_state=42)



In [11]:
train.shape, test.shape, val.shape

((17919, 12), (9600, 12), (4480, 12))

In [12]:
features = ['security_deposit', 'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights', 'city', 'state', 'space', 'description', 'room_type']
target = 'price'

In [13]:

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
X_test = test[target]

In [9]:
from lightgbm import LGBMClassifier
import category_encoders as ce
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.OneHotEncoder(),
    SimpleImputer(),
    StandardScaler(), 
    LGBMClassifier()
    
)

# Fit on train, score on val, predict on test
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))
y_pred = pipeline.predict(X_test)


In [ ]:
from sklearn.externals import joblib
joblib.dump(pipeline, 'placeholder.pkl')